In [1]:
import ase
import math

from ase.io import read as ase_read
from itertools import permutations
import numpy as np
import copy

In [4]:
def get_replace_atomic_numbers(compound, target_atomic_numbers):
    """
    get atomic numbers replaced list from origin one to `target_atomic_numbers` with full permutations.

    (e.g, origin: [1,1,1,1,2,2], target_atomic_numbers:[3,4], result:[[3,3,3,3,4,4],[4,4,4,4,3,3]])

    (e.g, origin: [1,2,2,2,2,3,3], target_atomic_numbers:[4,5,6], result:[[4,5,5,5,5,6,6],[4,6,6,6,6,5,5],[5,4,4,4,4,6,6],[5,6,6,6,6,4,4],[6,5,5,5,5,4,4],[6,4,4,4,4,5,5]])
    """
    target_atomic_numbers = list(set(target_atomic_numbers))

    # full permutations of target_atomic_numbers
    targets = [list(i) for i in permutations(target_atomic_numbers, len(target_atomic_numbers))]
    # print("targets", targets)

    # numpy atomic numbers from original compound
    np_atomic_numbers = compound.get_atomic_numbers()
    set_atomic_numbers = [i for i in set(np_atomic_numbers)]
    # print(np_atomic_numbers)
    # print("set_atomic_numbers", set_atomic_numbers)

    replace_atomic_numbers = []
    for j, target in enumerate(targets):
        array = np.array(np_atomic_numbers)
        # final_array = np.zeros_like(array)
        # # print("================")
        # # print("原始array", array)
        # for i, d in enumerate(set_atomic_numbers):
        #     # print("原始件", d)
        #     # print("替换件:", target[i])
        #     final_array[array == d] = target[i]
        #     # print("array", final_array)
        # # print("最终array"final_array)
        # replace_atomic_numbers.append(final_array)
        replace_atomic_numbers.append(array)
        
    print(np_atomic_numbers)
    print(replace_atomic_numbers)

    return replace_atomic_numbers


def scale_compound_volume(compound, scaling_factor):
    """
    scale the compound by volume
    """
    scaling_factor = scaling_factor ** (1 / 3)
    compound.set_cell(compound.cell * scaling_factor, scale_atoms=True)

In [5]:
# 354 355
# 134 130
compound = ase_read("../../dataset/raw/CONFIG_1.poscar", format="vasp")
scales = [0.96, 0.98, 1.00, 1.02, 1.04]
atomic_numbers = [58, 27, 29]
# print("original pos", compound.get_positions())
print("origin one,", "volume:", round(compound.get_volume(), 2), compound)

# get scaled compounds
scaled_compounds = []
for i, d in enumerate(scales):
    sacled_compound = copy.deepcopy(compound)
    scale_compound_volume(sacled_compound, d)
    scaled_compounds.append(sacled_compound)
print("num hypothesis scaled compounds:", len(scaled_compounds))

# get atomic numbers
hypothesis_atomic_numbers = get_replace_atomic_numbers(compound, atomic_numbers)
print("num hypothesis atomic numbers group:", len(hypothesis_atomic_numbers))

# get hypothesis compounds
hypothesis_compounds = []
for i, sacled_compound in enumerate(scaled_compounds):
    for j, hypothesis_atomic_number in enumerate(hypothesis_atomic_numbers):
        compound = copy.deepcopy(sacled_compound)
        compound.set_atomic_numbers(hypothesis_atomic_number)
        hypothesis_compounds.append(compound)

print("total hypothesis for single origin compound: ", len(hypothesis_compounds))
[print("volume:", round(i.get_volume(), 2), i) for i in hypothesis_compounds]
# [print("compound position:", i.get_positions()) for i in hypothesis_compounds]

origin one, volume: 440.63 Atoms(symbols='Ac8Ag4Ir4', pbc=True, cell=[[7.60953026, 0.0, 5e-16], [-5e-16, 7.60953026, 5e-16], [0.0, 0.0, 7.60953026]])
num hypothesis scaled compounds: 5
[89 89 89 89 89 89 89 89 47 47 47 47 77 77 77 77]
[array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77]), array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77]), array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77]), array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77]), array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77]), array([89, 89, 89, 89, 89, 89, 89, 89, 47, 47, 47, 47, 77, 77, 77, 77])]
num hypothesis atomic numbers group: 6
total hypothesis for single origin compound:  30
volume: 423.0 Atoms(symbols='Ac8Ag4Ir4', pbc=True, cell=[[7.506686162878032, 0.0, 4.932424148660941e-16], [-4.932424148660941e-16, 7.506686162878032, 4.932424148660941e-16], [0.0, 0.0, 7.506686162878032]])
volume: 423.0 Atoms(symbols='Ac8

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

---

In [9]:
print(hypothesis_compounds[28].get_atomic_numbers())
print(hypothesis_compounds[29].get_atomic_numbers())

[29 29 29 29 29 29 29 29 27 27 27 27 58 58 58 58]
[29 29 29 29 29 29 29 29 58 58 58 58 27 27 27 27]


In [10]:
print(hypothesis_compounds[28])
print(hypothesis_compounds[28].positions)
print(hypothesis_compounds[28].get_volume())
print(hypothesis_compounds[28].get_all_distances(mic=True))

Atoms(symbols='Cu8Co4Ce4', pbc=True, cell=[[7.709667141573198, 0.0, 5.065797019100886e-16], [-5.065797019100886e-16, 7.709667141573198, 5.065797019100886e-16], [0.0, 0.0, 7.709667141573198]])
[[ 1.92741679e+00  5.78225036e+00  1.92741679e+00]
 [ 5.78225036e+00  5.78225036e+00  1.92741679e+00]
 [ 1.92741679e+00  1.92741679e+00  5.78225036e+00]
 [ 5.78225036e+00  1.92741679e+00  5.78225036e+00]
 [ 5.78225036e+00  5.78225036e+00  5.78225036e+00]
 [ 1.92741679e+00  5.78225036e+00  5.78225036e+00]
 [ 5.78225036e+00  1.92741679e+00  1.92741679e+00]
 [ 1.92741679e+00  1.92741679e+00  1.92741679e+00]
 [-2.53289851e-16  3.85483357e+00  2.53289851e-16]
 [ 0.00000000e+00  0.00000000e+00  3.85483357e+00]
 [ 3.85483357e+00  3.85483357e+00  3.85483357e+00]
 [ 3.85483357e+00  0.00000000e+00  2.53289851e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.53289851e-16  3.85483357e+00  3.85483357e+00]
 [ 3.85483357e+00  0.00000000e+00  3.85483357e+00]
 [ 3.85483357e+00  3.85483357e+00  5.065797

In [4]:
a = ase_read("./2.poscar", format="vasp")
print(a)
print(a.positions)
print(a.get_volume())
print(a.get_all_distances(mic=True))

Atoms(symbols='Cu8Co4Ce4', pbc=True, cell=[[7.709667141573196, 0.0, 5.065797019100885e-16], [-5.065797019100885e-16, 7.709667141573196, 5.065797019100885e-16], [0.0, 0.0, 7.709667141573196]])
[[ 1.92741679e+00  5.78225036e+00  1.92741679e+00]
 [ 5.78225036e+00  5.78225036e+00  1.92741679e+00]
 [ 1.92741679e+00  1.92741679e+00  5.78225036e+00]
 [ 5.78225036e+00  1.92741679e+00  5.78225036e+00]
 [ 5.78225036e+00  5.78225036e+00  5.78225036e+00]
 [ 1.92741679e+00  5.78225036e+00  5.78225036e+00]
 [ 5.78225036e+00  1.92741679e+00  1.92741679e+00]
 [ 1.92741679e+00  1.92741679e+00  1.92741679e+00]
 [-2.53289851e-16  3.85483357e+00  2.53289851e-16]
 [ 0.00000000e+00  0.00000000e+00  3.85483357e+00]
 [ 3.85483357e+00  3.85483357e+00  3.85483357e+00]
 [ 3.85483357e+00  0.00000000e+00  2.53289851e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.53289851e-16  3.85483357e+00  3.85483357e+00]
 [ 3.85483357e+00  0.00000000e+00  3.85483357e+00]
 [ 3.85483357e+00  3.85483357e+00  5.065797